In [35]:
# importing libraries
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.ops import sigmoid
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageDraw, ImageFont
import numpy as np

In [36]:
# loading model 
model = load_model('best_model_new.keras')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
# dimentions of the image
img_width, img_height = 150, 150
batch_size = 32

# preparing 
test_dir = './datasets/cats-dogs/test/'

test_datagen = ImageDataGenerator(
    1. / 255
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 700 images belonging to 2 classes.


In [39]:

# evaluating the model
loss, accuracy = model.evaluate(test_generator)
print(f"Loss:{loss:.4f}, Accuracy:{accuracy*100:.2f}%")


22/22 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.8986 - loss: 31.1535
Loss:39.9891, Accuracy:87.57%


In [40]:
# predict the image 
img = image.load_img('.//datasets/cats-dogs/test/dogs/27.jpg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images)
score = float(sigmoid(classes[0][0]))
print(score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
0.7310585975646973


In [34]:
# drawing a bounbox on images
# loading an image 
image = Image.open('.//datasets/cats-dogs/test/dogs/27.jpg')

predictions = [
    # Sample predictions (x_min, y_min, width, height, confidence, class_id)
    [100, 150, 200, 250, 0.85, 1],
    #[300, 200, 150, 100, 0.90, 0],
]

# defining classes names
class_names = {
    0:"Cat", 
    1:"Dog"
}

# creating drowing context

draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

for pred in predictions:
    x_min, y_min, width, height, confidence, class_id = pred
    x_max, y_max = x_min + width, y_min + height

    draw.rectangle([x_min, y_min, x_max, y_max], outline='green', width=2)

    # adding labels
    label = f"{class_names[class_id]}: {confidence:.2f}"
    draw.text((x_min, y_min - 10), label, fill='green', font=font)

# save or show the image
image.save('predicted_image.jpg')